# X Crawler

With help of [this link](https://github.com/godkingjay/selenium-twitter-scraper).

In [1]:
import configparser
import os
import pandas as pd
import re
from hazm import Normalizer
from tqdm.notebook import tqdm

In [2]:
config = configparser.ConfigParser()
config.read('.env')
config = config['TWITTER']
config

<Section: TWITTER>

In [3]:
try:
    os.mkdir('tweets')
except FileExistsError:
    print('tweets directory already exists')
os.chdir('selenium-twitter-scraper')

base_command = f'python scraper --user=@{config["TWITTER_USERNAME"]} --password={config["TWITTER_PASSWORD"]} --top '
os.system(base_command + '--tweets 5 --hashtag "فمنیست"')
os.system(base_command + '--tweets 5 --query "فقط زن"')
os.system(base_command + '--tweets 5 --query "فقط مرد"')

os.system('cp -r tweets ../')
os.chdir('..')

tweets directory already exists
Loading .env file
Loaded .env file


Initializing Twitter Scraper...
Setup WebDriver...
Initializing FirefoxDriver...
WebDriver Setup Complete

Logging in to Twitter...

Login Successful

Scraping Top Tweets from #فمنیست...
Progress: [[========================================]] 100.00% 5 of 5                                                  
Scraping Complete
Tweets: 5 out of 5

Saving Tweets to CSV...
CSV Saved: ./tweets/2024-07-18_15-31-52_tweets_1-5.csv
Loading .env file
Loaded .env file


Initializing Twitter Scraper...
Setup WebDriver...
Initializing FirefoxDriver...
WebDriver Setup Complete

Logging in to Twitter...

Login Successful

Scraping Top Tweets from فقط زن search...
Progress: [[========================================]] 100.00% 5 of 5                                                  
Scraping Complete
Tweets: 5 out of 5

Saving Tweets to CSV...
CSV Saved: ./tweets/2024-07-18_15-32-28_tweets_1-5.csv
Loading .env file
Loaded .env file


Ini

In [9]:
tweets_files = os.listdir('tweets')
tweets_files = [tweets_file for tweets_file in tweets_files if tweets_file.endswith('.csv')]
tweets_files = sorted(tweets_files)
tweets_files

['2024-07-18_15-31-52_tweets_1-5.csv',
 '2024-07-18_15-32-28_tweets_1-5.csv',
 '2024-07-18_15-33-03_tweets_1-5.csv']

In [10]:
tweets = []

for tweets_file in tweets_files:
    current_file = pd.read_csv(f'tweets/{tweets_file}')
    current_file = current_file['Content'].tolist()
    tweets += current_file

tweets

['زنان #فمنیست',
 'عرض ادب به #فمنیست های از «تورک بودن خود شرمنده» آزربایجان ',
 'تیپیکال #لیبرال و #فمنیست و #چپ...!',
 '#حقوق_زنان در فرقه منحوس #رجوی#فمنیست ها و مدعیان حقوق زن چرا صداشون در نمیاد#Palestine',
 'پاچه گرفتن از صفات بارز #فمنیست هاست ',
 'حالا پس از یکسال،علینژاد نه تنها انصراف نداده بلکه با تاسیس یک موسسه بهمراه نازنین بنیادی و زکا و چند تن دیگر،راه دریافت غرامت را سهل نموده تا نشان دهد، #زن_زندگى_آزادى فقط یک کاسبی بود و بس\nمعلوم نیست چی میزنه این ریختی شده',
 'خونه بدون زن این شکلیه، فقط کافیه\u200c تمیزگونه زندگی کنیم!',
 'فقط یک زن می\u200cتواند اینقدر مهربان باشد!\n\nزنان هزارستان باید سطح جهان احترام و حرمت شود.\n\nمادر\u200c معنوی مهاجران. تولدت مبارک خانوم مرکل',
 '۲۱ نکته Dark روان زنانرشتو۱- گریه کردن مقابل یک دختر مانند خونریزی در مقابل کوسه است ۲- اگر میخواید یک دختر بشناسید به رفتارش نگاه کنید نه گفتارش ( چون اکثر اوقات دروغ میگن ) \n۳-اگر میخواید در بحث و جدل با یک زن پیروز بشید فقط سکوت کنید و نتیجه حیرت انگیزش ببینید',
 'خواستین بفهمین برای یه زن چقد

In [15]:
# pd.DataFrame(tweets).to_csv('tweets/tweets.csv', index=False)

In [16]:
# tweets = pd.read_csv('tweets/tweets.csv')['0'].tolist()
# tweets

In [17]:
with open('stopwords/chars.txt', 'r', encoding='utf-8') as file:
    chars_stop_words = ''.join(file.read().splitlines())

chars_stop_words = chars_stop_words.replace('[', '\[')
chars_stop_words = chars_stop_words.replace(']', '\]')
chars_stop_words += '\n\t\r'
chars_pattern = re.compile(f'[{chars_stop_words}]')
chars_pattern

re.compile(r'[\ufeff!"#()*,-./:\[\]«»،؛؟۰۱۲۳۴۵۶۷۸۹…$ًٌٍَُِّْءٰٔ﷼\n\t\r]',
           re.UNICODE)

In [18]:
# https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
emojis_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+")
emojis_pattern

re.compile(r'[😀-🙏🌀-🗿🚀-\U0001f6ff\U0001f1e0-🇿]+', re.UNICODE)

In [19]:
normalizer = Normalizer()

In [20]:
def preprocess_tweet(tweet):
    global chars_pattern, emojis_pattern, normalizer
    
    tweet = emojis_pattern.sub(r' ', tweet)
    tweet = chars_pattern.sub(r' ', tweet)
    tweet = normalizer.normalize(tweet)
    
    return tweet

In [21]:
progress_bar = tqdm(tweets)
preprocessed_tweets = []
for tweet in progress_bar:
    preprocessed_tweets.append(preprocess_tweet(tweet))
    
preprocessed_tweets

  0%|          | 0/15 [00:00<?, ?it/s]

['زنان فمنیست',
 'عرض ادب به فمنیست\u200cهای از تورک بودن خود شرمنده آزربایجان',
 'تیپیکال لیبرال و فمنیست و چپ',
 'حقوق_زنان در فرقه منحوس رجوی فمنیست\u200cها و مدعیان حقوق زن چرا صداشون در نمیاد Palestine',
 'پاچه گرفتن از صفات بارز فمنیست هاست',
 'حالا پس از یکسال علینژاد نه\u200cتنها انصراف نداده بلکه با تاسیس یک موسسه بهمراه نازنین بنیادی و زکا و چند تن دیگر راه دریافت غرامت را سهل نموده تا نشان دهد زن_زندگی_آزادی فقط یک کاسبی بود و بس معلوم نیست چی میزنه این ریختی شده',
 'خونه بدون زن این شکلیه فقط کافیه تمیزگونه زندگی کنیم',
 'فقط یک زن می\u200cتواند اینقدر مهربان باشد زنان هزارستان باید سطح جهان احترام و حرمت شود مادر معنوی مهاجران تولدت مبارک خانوم مرکل',
 'نکته Dark روان زنانرشتو گریه کردن مقابل یک دختر مانند خونریزی در مقابل کوسه است اگر میخواید یک دختر بشناسید به رفتارش نگاه کنید نه گفتارش چون اکثر اوقات دروغ میگن اگر میخواید در بحث و جدل با یک زن پیروز بشید فقط سکوت کنید و نتیجه حیرت انگیزش ببینید',
 'خواستین بفهمین برای یه زن چقدر مهمین به این توجه کنید\u200cکه زن\u200cها

# Remove Bias Using Llama

In [34]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import notebook_login

In [28]:
notebook_login()

In [38]:
model_name = 'meta-llama/Meta-Llama-3-8B'

In [39]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer

PreTrainedTokenizerFast(name_or_path='meta-llama/Meta-Llama-3-8B', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>'}, clean_up_tokenization_spaces=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)
model

model-00003-of-00004.safetensors:  27%|##6       | 1.32G/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]